In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils import PkdbModel, caffeine_idx, measurement_type_data, filter_out, convert_unit, unstring
plt.style.use('seaborn-whitegrid')
import numpy as np

{'token': '55388d17d53f133a3f7eb7c69dd0874731fe3445'}


In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
def to_numeric(df_data):
    df_data[("weight","value")] = pd.to_numeric(df_data[("weight","value")])
    df_data[("weight","mean")] = pd.to_numeric(df_data[("weight","mean")])
    df_data["value_intervention"] = pd.to_numeric(df_data["value_intervention"])
    df_data["median"] = pd.to_numeric(df_data["median"])
    df_data["mean"] = pd.to_numeric(df_data["mean"])
    df_data["value"] = pd.to_numeric(df_data["value"])
    df_data["se"] = pd.to_numeric(df_data["se"])
    df_data["sd"] = pd.to_numeric(df_data["sd"])
    df_data["cv"] = pd.to_numeric(df_data["cv"])
    return df_data
     
def infer_from_interventions(data, unit_abs = "gram"):
    unit_rel= unit_abs + " / kilogram"
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])
    data_rel["inferred"] = True
    data_abs["inferred"] = True

    
    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
    data_rel["inferred"] = True
    data_abs["inferred"] = True
    
    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    return data

def infer_from_outputs(data, unit_abs="liter / hour"):
    unit_rel= unit_abs + " / kilogram"
    data_rel = convert_unit(data,
                                 unit_in=unit_abs,
                                 unit_out=unit_rel, 
                                 factor=1.0/data[("weight","value")],
                                 unit_field="unit", 
                                 data_fields=['value'], 
                                 subset=[("weight","value"),"value"])

    data_abs_i = convert_unit(data,
                                 unit_in=unit_rel,
                                 unit_out=unit_abs, 
                                 factor=data[("weight","value")],
                                 unit_field="unit", 
                                 data_fields=['value'],
                                 subset=[("weight","value"),"value"])
    data_rel["inferred"] = True
    data_abs_i["inferred"] = True
    

    data = pd.concat([data,data_rel,data_abs_i], ignore_index=True)
    
    data_rel = convert_unit(data,
                             unit_in=unit_abs,
                             unit_out=unit_rel, 
                             factor=1.0/data[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se'],
                             subset=[("weight","mean"),"mean"])

    data_abs = convert_unit(data,
                             unit_in=unit_rel,
                             unit_out=unit_abs, 
                             factor=data[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se'],
                             subset=[("weight","mean"),"mean"])
    
    data_rel["inferred"] = True
    data_abs["inferred"] = True


    data = pd.concat([data,data_rel,data_abs], ignore_index=True)
    
    return data

In [4]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.data.loc[(all_complete.data[("sex","choice")] == "M"),("oral contraceptives","choice")] = "N"
complete_caffeine = all_complete.data[caffeine_idx(all_complete.data)]
complete_caffeine["inferred"] = False

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (15,25,28,29,30,44,72,115,117,119,121,123,126,127,128,129,132,133,134,136,146,155,157,158,159,168,169,170,171,178,180,181,182,183,185,186,188,190,192,198,199,200,207,208,212,215,216,224,229,233,234,239,243) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
caffeine_clearance = measurement_type_data(complete_caffeine,"clearance")
caffeine_clearance = filter_out(caffeine_clearance,"unit",["milliliter / meter ** 2 / minute"])

In [6]:
caffeine_clearance.groupby(["unit"]).apply(len)

unit
liter / hour               338
liter / hour / kilogram    405
dtype: int64

In [7]:
caffeine_clearance = to_numeric(caffeine_clearance)
caffeine_clearance = infer_from_interventions(caffeine_clearance,"gram")
caffeine_clearance = infer_from_outputs(caffeine_clearance,"liter / hour")
caffeine_clearance.groupby("inferred").apply(len)

inferred
False     743
True     1200
dtype: int64

In [8]:
caffeine_clearance = convert_unit(caffeine_clearance,unit_in="liter / hour",unit_out="ml/min", factor=1000.0/60,data_fields=['mean','median','sd', 'se','value'])
caffeine_clearance = convert_unit(caffeine_clearance,unit_in="gram",unit_out="mg", factor=1000.0,data_fields=['value_intervention'],unit_field="unit_intervention")

caffeine_clearance = convert_unit(caffeine_clearance,unit_in="liter / hour / kilogram",unit_out="ml/min/kg", factor=1000.0/60,data_fields=['mean','median','sd', 'se','value'])
caffeine_clearance = convert_unit(caffeine_clearance,unit_in="gram / kilogram",unit_out="mg/kg", factor=1000.0,data_fields=['value_intervention'],unit_field="unit_intervention")


In [9]:
caffeine_clr = PkdbModel("caffeine_clearance",destination="3-inferred")
caffeine_clr.data = caffeine_clearance
caffeine_clr.save()

In [10]:
caffeine_clearance.groupby(["unit", "unit_intervention"]).apply(len)

unit       unit_intervention
ml/min     mg                   570
           mg/kg                400
ml/min/kg  mg                   552
           mg/kg                421
dtype: int64

In [11]:
caffeine_clearance.groupby(["calculated"]).apply(len)

calculated
False    1737
True      206
dtype: int64

## Timecourses

In [12]:
caffeine_timecourse = measurement_type_data(complete_caffeine,"concentration")
caffeine_timecourse = filter_out(caffeine_timecourse,"output_type",["outputs"])


In [13]:
caffeine_timecourse.groupby(["unit","unit_intervention","inferred"]).count()

study  output_type   pk  access  \
unit         unit_intervention inferred                                    
gram / liter gram              False       110          110  110     110   
             gram / kilogram   False        31           31   31      31   

                                         measurement_type  tissue  \
unit         unit_intervention inferred                             
gram / liter gram              False                  110     110   
             gram / kilogram   False                   31      31   

                                         allowed_users  substance  \
unit         unit_intervention inferred                             
gram / liter gram              False               110        110   
             gram / kilogram   False                31         31   

                                         interventions  raw_pk  ...  \
unit         unit_intervention inferred                         ...   
gram / liter gram              False               110     110  ...   
             gram / kilogram   False                31      31  ...   

                                         (cyp2d6 variant, choice)  \
unit         unit_intervention inferred                             
gram / liter gram              False                            0   
             gram / kilogram   False                            0   

                                         (cyp2d6 variant, count)  \
unit         unit_intervention inferred                            
gram / liter gram              False                           0   
             gram / kilogram   False                           0   

                                         (lean body mass, count)  \
unit         unit_intervention inferred                            
gram / liter gram              False                           0   
             gram / kilogram   False                           0   

                                         (lean body mass, mean)  \
unit         unit_intervention inferred                           
gram / liter gram              False                          0   
             gram / kilogram   False                          0   

                                         (lean body mass, sd)  \
unit         unit_intervention inferred                         
gram / liter gram              False                        0   
             gram / kilogram   False                        0   

                                         (lean body mass, unit)  \
unit         unit_intervention inferred                           
gram / liter gram              False                          0   
             gram / kilogram   False                          0   

                                         (fat weight, count)  \
unit         unit_intervention inferred                        
gram / liter gram              False                       0   
             gram / kilogram   False                       8   

                                         (fat weight, mean)  (fat weight, se)  \
unit         unit_intervention inferred                                         
gram / liter gram              False                      0                 0   
             gram / kilogram   False                      8                 8   

                                         (fat weight, unit)  
unit         unit_intervention inferred                      
gram / liter gram              False                      0  
             gram / kilogram   False                      8  

[2 rows x 241 columns]

In [14]:
caffeine_timecourse = filter_out(caffeine_timecourse,"unit",["mole / liter"])
caffeine_timecourse.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  access  \
unit         unit_intervention                                    
gram / liter gram                 110          110  110     110   
             gram / kilogram       31           31   31      31   

                                measurement_type  tissue  allowed_users  \
unit         unit_intervention                                            
gram / liter gram                            110     110            110   
             gram / kilogram                  31      31             31   

                                substance  interventions  raw_pk  ...  \
unit         unit_intervention                                    ...   
gram / liter gram                     110            110     110  ...   
             gram / kilogram           31             31      31  ...   

                                (cyp2d6 variant, count)  \
unit         unit_intervention                            
gram / liter gram                                     0   
             gram / kilogram                          0   

                                (lean body mass, count)  \
unit         unit_intervention                            
gram / liter gram                                     0   
             gram / kilogram                          0   

                                (lean body mass, mean)  (lean body mass, sd)  \
unit         unit_intervention                                                 
gram / liter gram                                    0                     0   
             gram / kilogram                         0                     0   

                                (lean body mass, unit)  (fat weight, count)  \
unit         unit_intervention                                                
gram / liter gram                                    0                    0   
             gram / kilogram                         0                    8   

                                (fat weight, mean)  (fat weight, se)  \
unit         unit_intervention                                         
gram / liter gram                                0                 0   
             gram / kilogram                     8                 8   

                                (fat weight, unit)  inferred  
unit         unit_intervention                                
gram / liter gram                                0       110  
             gram / kilogram                     8        31  

[2 rows x 242 columns]

In [15]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.data = caffeine_timecourse
caffeine_tc.to_array()
caffeine_timecourse = caffeine_tc.data

In [16]:
caffeine_timecourse = infer_from_interventions(caffeine_timecourse,"gram")
caffeine_timecourse = infer_from_outputs(caffeine_timecourse,"gram / liter")

In [17]:
caffeine_timecourse.groupby(["unit","unit_intervention","time_unit"]).count()
caffeine_timecourse.groupby("inferred").apply(len)

inferred
False    141
True     372
dtype: int64

In [18]:
caffeine_tc = PkdbModel("caffeine_timecourse",destination="3-inferred")
caffeine_tc.data = caffeine_timecourse
caffeine_tc.save()

In [19]:
caffeine_thalf = measurement_type_data(complete_caffeine,"thalf")
caffeine_thalf = to_numeric(caffeine_thalf)
caffeine_thalf.groupby(["unit","unit_intervention"]).count()


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

study  output_type   pk  access  measurement_type  \
unit unit_intervention                                                      
hour gram                 440          440  440     440               440   
     gram / kilogram      103          103  103     103               103   

                        tissue  allowed_users  substance  interventions  \
unit unit_intervention                                                    
hour gram                  440            440        440            440   
     gram / kilogram       103            103        103            103   

                        raw_pk  ...  (cyp2d6 variant, count)  \
unit unit_intervention          ...                            
hour gram                  440  ...                        0   
     gram / kilogram       103  ...                        0   

                        (lean body mass, count)  (lean body mass, mean)  \
unit unit_intervention                                                    
hour gram                                     0                       0   
     gram / kilogram                          0                       0   

                        (lean body mass, sd)  (lean body mass, unit)  \
unit unit_intervention                                                 
hour gram                                  0                       0   
     gram / kilogram                       0                       0   

                        (fat weight, count)  (fat weight, mean)  \
unit unit_intervention                                            
hour gram                                 0                   0   
     gram / kilogram                     16                  16   

                        (fat weight, se)  (fat weight, unit)  inferred  
unit unit_intervention                                                  
hour gram                              0                   0       440  
     gram / kilogram                  16                  16       103  

[2 rows x 242 columns]

In [20]:
caffeine_thalf = infer_from_interventions(caffeine_thalf,"gram")
caffeine_thalf = infer_from_outputs(caffeine_thalf,"hour")

In [21]:
caffeine_thalf = convert_unit(caffeine_thalf,unit_in="hour",unit_out="hr", factor=1,data_fields=['mean','median','sd', 'se','value'])
caffeine_thalf = convert_unit(caffeine_thalf,unit_in="gram",unit_out="g", factor=1,data_fields=['value_intervention'],unit_field="unit_intervention")

caffeine_thalf = convert_unit(caffeine_thalf,unit_in="hour / kilogram",unit_out="hr/kg", factor=1,data_fields=['mean','median','sd', 'se','value'])
caffeine_thalf = convert_unit(caffeine_thalf,unit_in="gram / kilogram",unit_out="mg/kg", factor=1000.0,data_fields=['value_intervention'],unit_field="unit_intervention")


In [22]:
caffeine_thalf.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  access  measurement_type  \
unit  unit_intervention                                                      
hr    g                    522          522  522     522               522   
      mg/kg                412          412  412     412               412   
hr/kg g                    391          391  391     391               391   
      mg/kg                391          391  391     391               391   

                         tissue  allowed_users  substance  interventions  \
unit  unit_intervention                                                    
hr    g                     522            522        522            522   
      mg/kg                 412            412        412            412   
hr/kg g                     391            391        391            391   
      mg/kg                 391            391        391            391   

                         raw_pk  ...  (cyp2d6 variant, count)  \
unit  unit_intervention          ...                            
hr    g                     522  ...                        0   
      mg/kg                 412  ...                        0   
hr/kg g                     391  ...                        0   
      mg/kg                 391  ...                        0   

                         (lean body mass, count)  (lean body mass, mean)  \
unit  unit_intervention                                                    
hr    g                                        0                       0   
      mg/kg                                    0                       0   
hr/kg g                                        0                       0   
      mg/kg                                    0                       0   

                         (lean body mass, sd)  (lean body mass, unit)  \
unit  unit_intervention                                                 
hr    g                                     0                       0   
      mg/kg                                 0                       0   
hr/kg g                                     0                       0   
      mg/kg                                 0                       0   

                         (fat weight, count)  (fat weight, mean)  \
unit  unit_intervention                                            
hr    g                                   16                  16   
      mg/kg                               16                  16   
hr/kg g                                   16                  16   
      mg/kg                               16                  16   

                         (fat weight, se)  (fat weight, unit)  inferred  
unit  unit_intervention                                                  
hr    g                                16                  16       522  
      mg/kg                            16                  16       412  
hr/kg g                                16                  16       391  
      mg/kg                            16                  16       391  

[4 rows x 242 columns]

In [23]:
caffeine_thf = PkdbModel("caffeine_thalf",destination="3-inferred")
caffeine_thf.data = caffeine_thalf
caffeine_thf.save()

In [24]:
caffeine_tmax = measurement_type_data(complete_caffeine,"tmax")
caffeine_tmax = to_numeric(caffeine_tmax)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [25]:
caffeine_tmax.groupby(["unit","unit_intervention"]).count()



study  output_type   pk  access  measurement_type  \
unit unit_intervention                                                      
hour gram                 222          222  222     222               222   
     gram / kilogram       56           56   56      56                56   

                        tissue  allowed_users  substance  interventions  \
unit unit_intervention                                                    
hour gram                  222            222        222            222   
     gram / kilogram        56             56         56             56   

                        raw_pk  ...  (cyp2d6 variant, count)  \
unit unit_intervention          ...                            
hour gram                  222  ...                        0   
     gram / kilogram        56  ...                        0   

                        (lean body mass, count)  (lean body mass, mean)  \
unit unit_intervention                                                    
hour gram                                     0                       0   
     gram / kilogram                          0                       0   

                        (lean body mass, sd)  (lean body mass, unit)  \
unit unit_intervention                                                 
hour gram                                  0                       0   
     gram / kilogram                       0                       0   

                        (fat weight, count)  (fat weight, mean)  \
unit unit_intervention                                            
hour gram                                 0                   0   
     gram / kilogram                     16                  16   

                        (fat weight, se)  (fat weight, unit)  inferred  
unit unit_intervention                                                  
hour gram                              0                   0       222  
     gram / kilogram                  16                  16        56  

[2 rows x 242 columns]

In [26]:
caffeine_tmax = infer_from_interventions(caffeine_tmax)
caffeine_tmax = infer_from_outputs(caffeine_tmax, "hour")

caffeine_tmax = convert_unit(caffeine_tmax,unit_in="hour",unit_out="hr", factor=1,data_fields=['mean','median','sd', 'se','value'])
caffeine_tmax = convert_unit(caffeine_tmax,unit_in="gram",unit_out="g", factor=1,data_fields=['value_intervention'],unit_field="unit_intervention")

caffeine_tmax = convert_unit(caffeine_tmax,unit_in="hour / kilogram",unit_out="hr/kg", factor=1,data_fields=['mean','median','sd', 'se','value'])
caffeine_tmax = convert_unit(caffeine_tmax,unit_in="gram / kilogram",unit_out="mg/kg", factor=1000.0,data_fields=['value_intervention'],unit_field="unit_intervention")


In [27]:
caffeine_tmax.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  access  measurement_type  \
unit  unit_intervention                                                      
hr    g                    257          257  257     257               257   
      mg/kg                195          195  195     195               195   
hr/kg g                    174          174  174     174               174   
      mg/kg                174          174  174     174               174   

                         tissue  allowed_users  substance  interventions  \
unit  unit_intervention                                                    
hr    g                     257            257        257            257   
      mg/kg                 195            195        195            195   
hr/kg g                     174            174        174            174   
      mg/kg                 174            174        174            174   

                         raw_pk  ...  (cyp2d6 variant, count)  \
unit  unit_intervention          ...                            
hr    g                     257  ...                        0   
      mg/kg                 195  ...                        0   
hr/kg g                     174  ...                        0   
      mg/kg                 174  ...                        0   

                         (lean body mass, count)  (lean body mass, mean)  \
unit  unit_intervention                                                    
hr    g                                        0                       0   
      mg/kg                                    0                       0   
hr/kg g                                        0                       0   
      mg/kg                                    0                       0   

                         (lean body mass, sd)  (lean body mass, unit)  \
unit  unit_intervention                                                 
hr    g                                     0                       0   
      mg/kg                                 0                       0   
hr/kg g                                     0                       0   
      mg/kg                                 0                       0   

                         (fat weight, count)  (fat weight, mean)  \
unit  unit_intervention                                            
hr    g                                   16                  16   
      mg/kg                               16                  16   
hr/kg g                                   16                  16   
      mg/kg                               16                  16   

                         (fat weight, se)  (fat weight, unit)  inferred  
unit  unit_intervention                                                  
hr    g                                16                  16       257  
      mg/kg                            16                  16       195  
hr/kg g                                16                  16       174  
      mg/kg                            16                  16       174  

[4 rows x 242 columns]

In [28]:
caffeine_tmx = PkdbModel("caffeine_tmax",destination="3-inferred")
caffeine_tmx.data = caffeine_tmax
caffeine_tmx.save()

In [29]:

caffeine_vd = measurement_type_data(complete_caffeine,"vd")
caffeine_vd = to_numeric(caffeine_vd)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [30]:
caffeine_vd.groupby(["unit","unit_intervention"]).count()



study  output_type   pk  access  \
unit             unit_intervention                                    
liter            gram                 175          175  175     175   
                 gram / kilogram       11           11   11      11   
liter / kilogram gram                 155          155  155     155   
                 gram / kilogram       78           78   78      78   

                                    measurement_type  tissue  allowed_users  \
unit             unit_intervention                                            
liter            gram                            175     175            175   
                 gram / kilogram                  11      11             11   
liter / kilogram gram                            155     155            155   
                 gram / kilogram                  78      78             78   

                                    substance  interventions  raw_pk  ...  \
unit             unit_intervention                                    ...   
liter            gram                     175            175     175  ...   
                 gram / kilogram           11             11      11  ...   
liter / kilogram gram                     155            155     155  ...   
                 gram / kilogram           78             78      78  ...   

                                    (cyp2d6 variant, count)  \
unit             unit_intervention                            
liter            gram                                     0   
                 gram / kilogram                          0   
liter / kilogram gram                                     0   
                 gram / kilogram                          0   

                                    (lean body mass, count)  \
unit             unit_intervention                            
liter            gram                                     0   
                 gram / kilogram                          0   
liter / kilogram gram                                     0   
                 gram / kilogram                          0   

                                    (lean body mass, mean)  \
unit             unit_intervention                           
liter            gram                                    0   
                 gram / kilogram                         0   
liter / kilogram gram                                    0   
                 gram / kilogram                         0   

                                    (lean body mass, sd)  \
unit             unit_intervention                         
liter            gram                                  0   
                 gram / kilogram                       0   
liter / kilogram gram                                  0   
                 gram / kilogram                       0   

                                    (lean body mass, unit)  \
unit             unit_intervention                           
liter            gram                                    0   
                 gram / kilogram                         0   
liter / kilogram gram                                    0   
                 gram / kilogram                         0   

                                    (fat weight, count)  (fat weight, mean)  \
unit             unit_intervention                                            
liter            gram                                 0                   0   
                 gram / kilogram                      0                   0   
liter / kilogram gram                                 0                   0   
                 gram / kilogram                     16                  16   

                                    (fat weight, se)  (fat weight, unit)  \
unit             unit_intervention                                         
liter            gram                              0                   0   
                 gram / kilogram                   0                   0   
liter / kilogram gram      

In [31]:
caffeine_vd =  infer_from_interventions(caffeine_vd, "gram")
caffeine_vd =  infer_from_outputs(caffeine_vd, "liter")

In [32]:
caffeine_vd = convert_unit(caffeine_vd,unit_in="liter",unit_out="l", factor=1,data_fields=['mean','median','sd', 'se','value'])
caffeine_vd = convert_unit(caffeine_vd,unit_in="gram",unit_out="g", factor=1,data_fields=['value_intervention'],unit_field="unit_intervention")

caffeine_vd = convert_unit(caffeine_vd,unit_in="liter / kilogram",unit_out="l/kg", factor=1,data_fields=['mean','median','sd', 'se','value'])
caffeine_vd = convert_unit(caffeine_vd,unit_in="gram / kilogram",unit_out="mg/kg", factor=1000.0,data_fields=['value_intervention'],unit_field="unit_intervention")

In [33]:
caffeine_vd.groupby(["unit","unit_intervention"]).count()

study  output_type   pk  access  measurement_type  \
unit unit_intervention                                                      
l    g                    329          329  329     329               329   
     mg/kg                284          284  284     284               284   
l/kg g                    351          351  351     351               351   
     mg/kg                307          307  307     307               307   

                        tissue  allowed_users  substance  interventions  \
unit unit_intervention                                                    
l    g                     329            329        329            329   
     mg/kg                 284            284        284            284   
l/kg g                     351            351        351            351   
     mg/kg                 307            307        307            307   

                        raw_pk  ...  (cyp2d6 variant, count)  \
unit unit_intervention          ...                            
l    g                     329  ...                        0   
     mg/kg                 284  ...                        0   
l/kg g                     351  ...                        0   
     mg/kg                 307  ...                        0   

                        (lean body mass, count)  (lean body mass, mean)  \
unit unit_intervention                                                    
l    g                                        0                       0   
     mg/kg                                    0                       0   
l/kg g                                        0                       0   
     mg/kg                                    0                       0   

                        (lean body mass, sd)  (lean body mass, unit)  \
unit unit_intervention                                                 
l    g                                     0                       0   
     mg/kg                                 0                       0   
l/kg g                                     0                       0   
     mg/kg                                 0                       0   

                        (fat weight, count)  (fat weight, mean)  \
unit unit_intervention                                            
l    g                                   16                  16   
     mg/kg                               16                  16   
l/kg g                                   16                  16   
     mg/kg                               16                  16   

                        (fat weight, se)  (fat weight, unit)  inferred  
unit unit_intervention                                                  
l    g                                16                  16       329  
     mg/kg                            16                  16       284  
l/kg g                                16                  16       351  
     mg/kg                            16                  16       307  

[4 rows x 242 columns]

In [34]:
caffeine_vd_model = PkdbModel("caffeine_vd",destination="3-inferred")
caffeine_vd_model.data = caffeine_vd
caffeine_vd_model.save()

In [35]:
caffeine_auc_inf = measurement_type_data(complete_caffeine,"auc_inf")
caffeine_auc_inf = to_numeric(caffeine_auc_inf)
caffeine_auc_inf.groupby(["unit","unit_intervention"]).count()


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

study  output_type   pk  access  \
unit                unit_intervention                                    
gram * hour / liter gram                 243          243  243     243   
                    gram / kilogram       44           44   44      44   

                                       measurement_type  tissue  \
unit                unit_intervention                             
gram * hour / liter gram                            243     243   
                    gram / kilogram                  44      44   

                                       allowed_users  substance  \
unit                unit_intervention                             
gram * hour / liter gram                         243        243   
                    gram / kilogram               44         44   

                                       interventions  raw_pk  ...  \
unit                unit_intervention                         ...   
gram * hour / liter gram                         243     243  ...   
                    gram / kilogram               44      44  ...   

                                       (cyp2d6 variant, count)  \
unit                unit_intervention                            
gram * hour / liter gram                                     0   
                    gram / kilogram                          0   

                                       (lean body mass, count)  \
unit                unit_intervention                            
gram * hour / liter gram                                     0   
                    gram / kilogram                          0   

                                       (lean body mass, mean)  \
unit                unit_intervention                           
gram * hour / liter gram                                    0   
                    gram / kilogram                         0   

                                       (lean body mass, sd)  \
unit                unit_intervention                         
gram * hour / liter gram                                  0   
                    gram / kilogram                       0   

                                       (lean body mass, unit)  \
unit                unit_intervention                           
gram * hour / liter gram                                    0   
                    gram / kilogram                         0   

                                       (fat weight, count)  \
unit                unit_intervention                        
gram * hour / liter gram                                 0   
                    gram / kilogram                     15   

                                       (fat weight, mean)  (fat weight, se)  \
unit                unit_intervention                                         
gram * hour / liter gram                                0                 0   
                    gram / kilogram                    15                15   

                                       (fat weight, unit)  inferred  
unit                unit_intervention                                
gram * hour / liter gram                                0       243  
                    gram / kilogram                    15        44  

[2 rows x 242 columns]

In [36]:
caffeine_auc_inf =  infer_from_interventions(caffeine_auc_inf, "gram")
caffeine_auc_inf =  infer_from_outputs(caffeine_auc_inf, "gram * hour / liter")

In [37]:
caffeine_auc_inf.groupby(["unit","unit_intervention"]).count()


study  output_type   pk  \
unit                           unit_intervention                            
gram * hour / liter            gram                 280          280  280   
                               gram / kilogram      242          242  242   
gram * hour / liter / kilogram gram                 235          235  235   
                               gram / kilogram      235          235  235   

                                                  access  measurement_type  \
unit                           unit_intervention                             
gram * hour / liter            gram                  280               280   
                               gram / kilogram       242               242   
gram * hour / liter / kilogram gram                  235               235   
                               gram / kilogram       235               235   

                                                  tissue  allowed_users  \
unit                           unit_intervention                          
gram * hour / liter            gram                  280            280   
                               gram / kilogram       242            242   
gram * hour / liter / kilogram gram                  235            235   
                               gram / kilogram       235            235   

                                                  substance  interventions  \
unit                           unit_intervention                             
gram * hour / liter            gram                     280            280   
                               gram / kilogram          242            242   
gram * hour / liter / kilogram gram                     235            235   
                               gram / kilogram          235            235   

                                                  raw_pk  ...  \
unit                           unit_intervention          ...   
gram * hour / liter            gram                  280  ...   
                               gram / kilogram       242  ...   
gram * hour / liter / kilogram gram                  235  ...   
                               gram / kilogram       235  ...   

                                                  (cyp2d6 variant, count)  \
unit                           unit_intervention                            
gram * hour / liter            gram                                     0   
                               gram / kilogram                          0   
gram * hour / liter / kilogram gram                                     0   
                               gram / kilogram                          0   

                                                  (lean body mass, count)  \
unit                           unit_intervention                            
gram * hour / liter            gram                                     0   
                               gram / kilogram                          0   
gram * hour / liter / kilogram gram                                     0   
                               gram / kilogram                          0   

                                                  (lean body mass, mean)  \
unit                           unit_intervention                           
gram * hour / liter            gram                                    0   
                               gram / kilogram                         0   
gram * hour / liter / kilogram gram                                    0   
                               gram / kilogram                         0   

                                                  (lean body mass, sd)  \
unit                           unit_intervention                         
gram * hour / liter            gram                                  0   
                               gram / kilogram                       0   
gram * hour / liter / kilogram gram                                  0   
                               gram / kilogram        

In [38]:
caffeine_auc_inf = convert_unit(caffeine_auc_inf,unit_in="gram * hour / liter",unit_out="g*hr/l", factor=1,data_fields=['mean','median','sd', 'se','value'])
caffeine_auc_inf = convert_unit(caffeine_auc_inf,unit_in="gram",unit_out="g", factor=1,data_fields=['value_intervention'],unit_field="unit_intervention")

caffeine_auc_inf = convert_unit(caffeine_auc_inf,unit_in="gram * hour / liter / kilogram",unit_out="g*hr/l/kg", factor=1,data_fields=['mean','median','sd', 'se','value'])
caffeine_auc_inf = convert_unit(caffeine_auc_inf,unit_in="gram / kilogram",unit_out="mg/kg", factor=1000.0,data_fields=['value_intervention'],unit_field="unit_intervention")
caffeine_auc_inf.groupby(["unit","unit_intervention"]).count()


study  output_type   pk  access  \
unit      unit_intervention                                    
g*hr/l    g                    280          280  280     280   
          mg/kg                242          242  242     242   
g*hr/l/kg g                    235          235  235     235   
          mg/kg                235          235  235     235   

                             measurement_type  tissue  allowed_users  \
unit      unit_intervention                                            
g*hr/l    g                               280     280            280   
          mg/kg                           242     242            242   
g*hr/l/kg g                               235     235            235   
          mg/kg                           235     235            235   

                             substance  interventions  raw_pk  ...  \
unit      unit_intervention                                    ...   
g*hr/l    g                        280            280     280  ...   
          mg/kg                    242            242     242  ...   
g*hr/l/kg g                        235            235     235  ...   
          mg/kg                    235            235     235  ...   

                             (cyp2d6 variant, count)  (lean body mass, count)  \
unit      unit_intervention                                                     
g*hr/l    g                                        0                        0   
          mg/kg                                    0                        0   
g*hr/l/kg g                                        0                        0   
          mg/kg                                    0                        0   

                             (lean body mass, mean)  (lean body mass, sd)  \
unit      unit_intervention                                                 
g*hr/l    g                                       0                     0   
          mg/kg                                   0                     0   
g*hr/l/kg g                                       0                     0   
          mg/kg                                   0                     0   

                             (lean body mass, unit)  (fat weight, count)  \
unit      unit_intervention                                                
g*hr/l    g                                       0                   15   
          mg/kg                                   0                   15   
g*hr/l/kg g                                       0                   15   
          mg/kg                                   0                   15   

                             (fat weight, mean)  (fat weight, se)  \
unit      unit_intervention                                         
g*hr/l    g                                  15                15   
          mg/kg                              15                15   
g*hr/l/kg g                                  15                15   
          mg/kg                              15                15   

                             (fat weight, unit)  inferred  
unit      unit_intervention                                
g*hr/l    g                                  15       280  
          mg/kg                              15       242  
g*hr/l/kg g                                  15       235  
          mg/kg                              15       235  

[4 rows x 242 columns]

In [39]:
caffeine_auc_inf_model = PkdbModel("caffeine_auc_inf",destination="3-inferred")
caffeine_auc_inf_model.data = caffeine_auc_inf
caffeine_auc_inf_model.save()

In [40]:
complete_caffeine.groupby("measurement_type").count()

,study,output_type,pk,access,tissue,allowed_users,substance,interventions,raw_pk,unit,...,"(cyp2d6 variant, count)","(lean body mass, count)","(lean body mass, mean)","(lean body mass, sd)","(lean body mass, unit)","(fat weight, count)","(fat weight, mean)","(fat weight, se)","(fat weight, unit)",inferred
measurement_type,,,,,,,,,,,,,,,,,,,,,
auc_end,164,164,164,164,164,164,164,164,164,164,...,0,0,0,0,0,7,7,7,7,164
auc_inf,287,287,287,287,287,287,287,287,287,287,...,0,0,0,0,0,15,15,15,15,287
clearance,745,745,745,745,745,745,745,745,745,745,...,0,0,0,0,0,16,16,16,16,745
clearance_intrinsic,3,3,3,3,3,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,3
clearance_intrinsic_unbound,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,2
clearance_renal,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
clearance_unbound,7,7,7,7,7,7,7,7,7,7,...,0,0,0,0,0,0,0,0,0,7
cmax,284,284,284,284,284,284,284,284,284,284,...,0,0,0,0,0,16,16,16,16,284
concentration,193,193,193,193,193,193,193,193,193,193,...,0,10,10,10,10,8,8,8,8,193


In [41]:
caffeine_kel = measurement_type_data(complete_caffeine,"kel")
caffeine_kel = to_numeric(caffeine_kel)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [42]:
caffeine_kel.groupby(["unit","unit_intervention"]).count()


study  output_type   pk  access  \
unit       unit_intervention                                    
1 / minute gram                 142          142  142     142   
           gram / kilogram       69           69   69      69   

                              measurement_type  tissue  allowed_users  \
unit       unit_intervention                                            
1 / minute gram                            142     142            142   
           gram / kilogram                  69      69             69   

                              substance  interventions  raw_pk  ...  \
unit       unit_intervention                                    ...   
1 / minute gram                     142            142     142  ...   
           gram / kilogram           69             69      69  ...   

                              (cyp2d6 variant, count)  \
unit       unit_intervention                            
1 / minute gram                                     0   
           gram / kilogram                          0   

                              (lean body mass, count)  (lean body mass, mean)  \
unit       unit_intervention                                                    
1 / minute gram                                     0                       0   
           gram / kilogram                          0                       0   

                              (lean body mass, sd)  (lean body mass, unit)  \
unit       unit_intervention                                                 
1 / minute gram                                  0                       0   
           gram / kilogram                       0                       0   

                              (fat weight, count)  (fat weight, mean)  \
unit       unit_intervention                                            
1 / minute gram                                 0                   0   
           gram / kilogram                     16                  16   

                              (fat weight, se)  (fat weight, unit)  inferred  
unit       unit_intervention                                                  
1 / minute gram                              0                   0       142  
           gram / kilogram                  16                  16        69  

[2 rows x 242 columns]

In [43]:
caffeine_kel =  infer_from_interventions(caffeine_kel, "gram")
caffeine_kel =  infer_from_outputs(caffeine_kel, "1 / minute")

In [44]:
caffeine_kel = convert_unit(caffeine_kel,unit_in="1 / minute",unit_out="1/hr", factor=60,data_fields=['mean','median','sd', 'se','value'])
caffeine_kel = convert_unit(caffeine_kel,unit_in="gram",unit_out="g", factor=1,data_fields=['value_intervention'],unit_field="unit_intervention")

caffeine_kel = convert_unit(caffeine_kel,unit_in="1 / minute / kilogram",unit_out="1/hr/kg", factor=60,data_fields=['mean','median','sd', 'se','value'])
caffeine_kel = convert_unit(caffeine_kel,unit_in="gram / kilogram",unit_out="mg/kg", factor=1000.0,data_fields=['value_intervention'],unit_field="unit_intervention")
caffeine_kel.groupby(["unit","unit_intervention"]).count()


study  output_type   pk  access  measurement_type  \
unit    unit_intervention                                                      
1/hr    g                    204          204  204     204               204   
        mg/kg                203          203  203     203               203   
1/hr/kg g                    196          196  196     196               196   
        mg/kg                196          196  196     196               196   

                           tissue  allowed_users  substance  interventions  \
unit    unit_intervention                                                    
1/hr    g                     204            204        204            204   
        mg/kg                 203            203        203            203   
1/hr/kg g                     196            196        196            196   
        mg/kg                 196            196        196            196   

                           raw_pk  ...  (cyp2d6 variant, count)  \
unit    unit_intervention          ...                            
1/hr    g                     204  ...                        0   
        mg/kg                 203  ...                        0   
1/hr/kg g                     196  ...                        0   
        mg/kg                 196  ...                        0   

                           (lean body mass, count)  (lean body mass, mean)  \
unit    unit_intervention                                                    
1/hr    g                                        0                       0   
        mg/kg                                    0                       0   
1/hr/kg g                                        0                       0   
        mg/kg                                    0                       0   

                           (lean body mass, sd)  (lean body mass, unit)  \
unit    unit_intervention                                                 
1/hr    g                                     0                       0   
        mg/kg                                 0                       0   
1/hr/kg g                                     0                       0   
        mg/kg                                 0                       0   

                           (fat weight, count)  (fat weight, mean)  \
unit    unit_intervention                                            
1/hr    g                                   16                  16   
        mg/kg                               16                  16   
1/hr/kg g                                   16                  16   
        mg/kg                               16                  16   

                           (fat weight, se)  (fat weight, unit)  inferred  
unit    unit_intervention                                                  
1/hr    g                                16                  16       204  
        mg/kg                            16                  16       203  
1/hr/kg g                                16                  16       196  
        mg/kg                            16                  16       196  

[4 rows x 242 columns]

In [45]:




caffeine_kel_model = PkdbModel("caffeine_kel",destination="3-inferred")
caffeine_kel_model.data = caffeine_kel
caffeine_kel_model.save()

In [46]:
#caffeine_vd[caffeine_vd["group_pk"] == 905][["study","group_count","group_pk","mean","unit","unit_intervention",'weight']]